In [37]:
# Built upon this blopgpost: https://blog.otoro.net/2015/11/24/mixture-density-networks-with-tensorflow/

import glob, os
%matplotlib inline

import numpy as np
import IPython
from IPython.display import HTML
from itertools import chain
import random

import tensorflow as tf

import pickle
import re
from matplotlib.ticker import FormatStrFormatter
from scipy.stats import norm
from scipy.stats import binned_statistic

import multiprocessing as mp
import math as m

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib 
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

from joblib import Parallel, delayed, dump, load

In [38]:
pathSave = "."

In [39]:
class getData:
    def __init__(self, _paraVecIndices, _xparaVecIndices, _tProfBounds, _trainPercent, _pathSave, _noiseLevel=0.):
        self.paraVecIndices = _paraVecIndices
        self.xparaVecIndices = _xparaVecIndices
        self.tProfBounds = _tProfBounds
        self.trainPercent = _trainPercent
        self.pathSave = _pathSave
        self.noiseLevel = _noiseLevel
        
    def visualizeData(self, picID, fontSize, Dict_baseline = {}, plotTProf=True, plotBaseline=False): 
        x_data, x_test, x_cv, y_data, y_test, y_cv, paraVec, xparaVec, rProf, startPoint, endPoint, indexer \
        = self.x_data, self.x_test, self.x_cv, self.y_data, self.y_test, self.y_cv, self.paraVec, self.xparaVec, \
            self.rProf, self.startPoint, self.endPoint, self.indexer    
                
        print('Added noise ' + str(self.noiseLevel))
        print("X_train shape: " + str(x_data.shape))
        print("Y_train shape: " + str(y_data.shape))
        print("X_test shape: " + str(x_test.shape))
        print("Y_test shape: " + str(y_test.shape))
        print("X_cv shape: " + str(x_cv.shape))
        print("Y_cv shape: " + str(y_cv.shape))
        print()
        
        alphaVal = 0.1
        tProfBoolean = False

        if 'Tprof_4p5Gyr' in xparaVec:
            xparaVec = xparaVec[0:-1]
            tProfBoolean = True

        for ind in range(np.size(paraVec)):
            fig, ax = plt.subplots()
            n, bins, patches = ax.hist(y_data[:, ind], 100, density=1)
            ax.set_xlabel(paraVec[ind])
            fig = plt.figure(figsize=(14,4), dpi=200)
            plt.subplots_adjust(top=0.92, bottom=0.1, left=0.05, right=0.95, hspace=0.10,wspace=0.35)
            nCols = np.size(xparaVec)
            nRows = 1
            for indX in range(nCols):
                ax = fig.add_subplot(nRows,nCols,indX+1)
                ax.plot(x_data[:,indX], y_data[:, ind],'ro', alpha=alphaVal)
                if plotBaseline:
                    ax.plot(x_data[:,indX],Dict_baseline['mu_var' + str(indX)] \
                                 +Dict_baseline['variance_var' + str(indX)],'k.')
                    ax.plot(x_data[:,indX],Dict_baseline['mu_var' + str(indX)] \
                                 -Dict_baseline['variance_var' + str(indX)],'k.')
                ax.yaxis.labelpad = 0.8
                ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
                plt.title("train")
                plt.xlabel(xparaVec[indX])
                plt.ylabel(paraVec[ind])
            plt.show()
            #fig.savefig(pathSave + "inverseProblemPics/" + "Input0_fig" + str(picID) + ".pdf", bbox_inches='tight')
            
        if  tProfBoolean and plotTProf:   
            length = int(np.size(paraVec))
            fig = plt.figure(figsize=(length,4), dpi=200)
            plt.subplots_adjust(top=0.9, bottom=0.1, left=0.05, right=0.95, hspace=0.10,wspace=0.3)
            nCols = np.size(paraVec)
            nRows = 2
            for indY in range(nCols):
                vecInterest = y_data[:, indY]
                colorsVec= [plt.cm.jet(i) for i in vecInterest]
                ax = fig.add_subplot(nRows,nCols,indY+1)
                ax.set_prop_cycle('color',colorsVec)
                for i, ii in enumerate(indexer):
                    rProf = self.rProf[startPoint:endPoint] # profiles['Dict_Rprof_4p5Gyr' + str(ii)]
                    rProf = (rProf-rProf.min())/(rProf.max()-rProf.min())
                    tProf = x_data[i,np.size(xparaVec):]
                    ax.plot(tProf, rProf, linewidth=0.5)
                ax.yaxis.labelpad = 0.8
                ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
                if indY > 0:
                    ax.axes.get_yaxis().set_ticks([])
                ax.set_title(paraVec[indY])
                ax.set_xlabel("T") #[K]")
                ax.set_ylabel("R") #[km] ")
                for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                     ax.get_xticklabels() + ax.get_yticklabels()):
                    item.set_fontsize(12)
                #ax.set_xlim([250,2250])
                #ax.set_ylim([1700,3400])
            plt.show()
           # fig.savefig(pathSave + "inverseProblemPics/" + "Input1_fig" + str(picID) + ".pdf", bbox_inches='tight')

In [40]:
class plotResults:
    def __init__(self, data, ax, ax1, alphaVal, title, _y_actual, Dict_prob_paras ,indPara,\
                 Dict_baseline, numParameters, kernel, plotIndividualPDFs=False):
        
        if "Test" in title:
            self.Dict_Test = Dict_prob_paras
        
        rho_m  = 3500. 
        g     = 3.7 
        alpha_m = 2.5e-5
        T_delta = 2000. 
        D = 1700e+3         
        k_diffusive = 1e-6 
        R = 8.314   
        
        def format_func(_val, tick_number):
            f = mticker.ScalarFormatter(useOffset=False, useMathText=True)
            _g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.2e' % x))
            fmt = mticker.FuncFormatter(_g)
            return "{}".format(fmt(_val))

        def dimensionalize(_val,_ind,isVariance=False):
            _min = data.pMin[_ind]
            _max = data.pMax[_ind]
            _val = _val*(_max-_min) 
            if not isVariance:
                _val = _val + _min
            
            if "Ra" in title and not isVariance:
                _val = np.log10(rho_m * g * alpha_m * T_delta * np.power(D,3.)/(np.power(10.,_val) * k_diffusive))
            
            if "ERef" in title:
                _val = _val*(R * T_delta)
            
            if "VRef" in title:
                _val = _val*(R * T_delta) /(rho_m * g * D)
            
            if "iniTempTop" in title:
                _val = _val*2000 
                if not isVariance:
                    _val = _val + 250
                    
            return _val

        xx = dimensionalize(_y_actual,indPara)    
        variance = []
        combined_mdn_std = []
        combined_mdn_mean = []
        xxSorted = np.sort(xx)        
        
        xxSize = np.size(xx)
        plotEvery = int(np.floor(xxSize*0.1))
        xxSorted.shape = (xxSize,1)
        Pr_xx = np.zeros((xxSize,xxSize))
        Pr_baseline = np.zeros((xxSize,xxSize))
        colors = ['r', 'g', 'm']
        
        def pdf(x):
            return 1./np.sqrt(2.*np.pi) * np.exp(-x**2/2.)
        #def cdf(x):
        #    return (1. + erf(x/np.sqrt(2))) / 2.
        def SSkew(x,a,e,w):
            t = (x-e) / w
            return 2. / w * pdf(t) * cdf(a*t)
        def getGamma(x):
            return m.gamma(x)
        def tf_beta(y, alpha, beta):
            Z = np.divide(np.multiply(getGamma(alpha),getGamma(beta)), getGamma(alpha + beta))
            result = np.divide(np.multiply(np.power(y,(alpha - 1.)), np.power((1. - y),(beta - 1.))),Z)
            return  result
             
        if plotIndividualPDFs:
            fig3 = plt.figure(figsize=(14,18))
           # plt.subplots_adjust(top=0.90, bottom=0.08, left=0.10, right=0.95, hspace=0.3,wspace=0.4)
            nRows = np.ceil((xxSize/plotEvery)/3)
            nCols = 3
            plotCounter = 1
        if kernel=='skewed_gaussian':
            out_skew_test = np.asarray(Dict_prob_paras["skew"+str(ind)])
            out_sigma_test = np.asarray(Dict_prob_paras["sigma"+str(ind)])
            out_mu_test = np.asarray(Dict_prob_paras["mu"+str(ind)])
            out_pi_test = np.asarray(Dict_prob_paras["pi"+str(ind)])
            for ind,val in enumerate(xxSorted):
                index = np.where(val == xx)[0][0]
                muIntermediate = []
                piIntermediate = []
                sigmaIntermediate = []
                skewIntermediate = []
                for i in range(out_sigma_test.shape[1]):
                    muIntermediate.append(out_mu_test[index,i])
                    sigmaIntermediate.append(out_sigma_test[index,i])
                    skewIntermediate.append(out_skew_test[index,i])
                    piIntermediate.append(out_pi_test[index,i])
                #mu = np.sum(np.asarray(piIntermediate) * np.asarray(muIntermediate))
                #sigma = np.sum(np.asarray(piIntermediate) * np.asarray(sigmaIntermediate))
                #skew = np.sum(np.asarray(piIntermediate) * np.asarray(skewIntermediate))
                pr = piIntermediate[0] * SSkew(xxSorted,skewIntermediate[0],muIntermediate[0],sigmaIntermediate[0])
                pr.shape = (pr.shape[0])
                for i in range(1,out_sigma_test.shape[1]):
                    prI = SSkew(xxSorted,skewIntermediate[i],muIntermediate[i],sigmaIntermediate[i])
                    prI.shape = (prI.shape[0])
                    pr += piIntermediate[i] * prI
                Pr_xx[ind,:] = pr

                if  plotIndividualPDFs and ind%plotEvery == 0:
                    ax3 = fig3.add_subplot(nRows,nCols,plotCounter) 
                    plotCounter += 1
                    legendStr = []
                    for i in range(np.size(muIntermediate)):
                        if piIntermediate[i] >= np.max(piIntermediate)*1e-15:
                            yPDF = SSkew(xxSorted,skewIntermediate[i],muIntermediate[i],sigmaIntermediate[i])
                            ax3.plot(xxSorted,yPDF) #/np.max(yPDF))
                            legendStr.append("%.4f" % piIntermediate[i])
                            #print(np.trapz(yPDF[:,0], xxSorted[:,0]))
                    #Pr_w = SSkew(xxSorted,skew,mu,sigma)
                    ax3.plot(xxSorted, Pr_xx[ind,:],"--")
                    print(np.trapz(Pr_xx[ind,:], xxSorted[:,0]))
                    ax3.plot(val,max(Pr_xx[ind,:]), "kx")
                    ax3.legend(legendStr)
                    
                
        elif kernel=='gaussian':
            out_pi_test = np.asarray(Dict_prob_paras["pi"+str(indPara)])
            out_sigma_test = np.asarray(Dict_prob_paras["sigma"+str(indPara)])
            out_mu_test = np.asarray(Dict_prob_paras["mu"+str(indPara)])
            for ind,val in enumerate(xxSorted):
                index = np.where(val == xx)[0][0]
                muIntermediate = []
                piIntermediate = []
                sigmaIntermediate = []
                for i in range(out_sigma_test.shape[1]):
                    muIntermediate.append(dimensionalize(out_mu_test[index,i],indPara))
                    sigmaIntermediate.append(dimensionalize(out_sigma_test[index,i],indPara,True))
                    piIntermediate.append(out_pi_test[index,i])
                    
                #mu = np.sum(np.asarray(piIntermediate) * np.asarray(muIntermediate))
                #sigma = np.sum(np.asarray(piIntermediate) * np.asarray(sigmaIntermediate))
                pr = piIntermediate[0] * norm.pdf(xxSorted, muIntermediate[0], sigmaIntermediate[0])
                for i in range(1,out_sigma_test.shape[1]):
                    pr += piIntermediate[i] * norm.pdf(xxSorted, muIntermediate[i], sigmaIntermediate[i])
                pr.shape = (pr.shape[0],)
                
                mean_mdn = 0
                for i in range(out_sigma_test.shape[1]):
                    mean_mdn += piIntermediate[i]*muIntermediate[i]
                var_mdn = 0
                for i in range(out_sigma_test.shape[1]):
                    var_mdn += piIntermediate[i]*(np.power(sigmaIntermediate[i],2) + np.power(muIntermediate[i],2) - np.power(mean_mdn,2))
                combined_mdn_std.append(np.power(var_mdn,0.5))
                combined_mdn_mean.append(mean_mdn)
                
                Pr_xx[ind,:] = pr #norm.pdf(xxSorted, mu, sigma)
                if  plotIndividualPDFs and ind%plotEvery == 0:
                    ax3 = fig3.add_subplot(nRows,nCols,plotCounter) 
                    plotCounter += 1
                    legendStr = []
                    for i in range(np.size(muIntermediate)):
                        if piIntermediate[i] >= np.max(piIntermediate)*1e-15:
                            yPDF = piIntermediate[i] * norm.pdf(xxSorted, muIntermediate[i], sigmaIntermediate[i])
                            ax3.plot(xxSorted,yPDF,colors[i]+'--', linewidth=2.5)
                            legendStr.append("%.4f" % piIntermediate[i])
                            if "Test" in title:
                                self.Dict_Test["forFig_" + title + "_pdf_mixture" + str(i) + "_case_" + str(ind)] = yPDF
                    #Pr_w = norm.pdf(xxSorted, mu, sigma)
                    ax3.plot(xxSorted, Pr_xx[ind,:],"k-", linewidth=2.5)
                    ax3.plot([val, val],[0,max(Pr_xx[ind,:])], "-", color='grey', linewidth=2.5)
                    
                    if "Test" in title:
                        self.Dict_Test["forFig_" + title + "_trueVal_" + str(ind)] = val
                    
                    #ax3.plot([combined_mdn_mean[ind]+combined_mdn_std[ind], combined_mdn_mean[ind]+combined_mdn_std[ind]],[0,max(Pr_xx[ind,:])], "k--", color='grey', linewidth=2.5)
                    #ax3.plot([combined_mdn_mean[ind]-combined_mdn_std[ind], combined_mdn_mean[ind]-combined_mdn_std[ind]],[0,max(Pr_xx[ind,:])], "k--", color='grey', linewidth=2.5)
                    for item in ([ax3.title, ax3.xaxis.label, ax3.yaxis.label] +
                     ax3.get_xticklabels() + ax3.get_yticklabels()):
                        item.set_fontsize(20)
                    if "ERef" in title:
                        ax3.set_xticks([1e+5,3e+5,5e+5])
                        #ax3.set_yticks([1e+5,2e+5,3e+5,4e+5,5e+5])
                        ax3.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
                        ax3.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
                    elif "VRef" in title:
                        ax3.set_xticks([4e-6, 7e-6, 10e-6])
                        #ax3.set_yticks([4e-6, 5e-6, 6e-6, 7e-6, 8e-6, 9e-6, 10e-6])
                        ax3.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
                        ax3.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
                    plt.tight_layout()

                    #ax3.legend(legendStr)
        elif kernel=='beta':
            out_pi_test = np.asarray(Dict_prob_paras["pi"+str(ind)])
            out_alpha_test = np.asarray(Dict_prob_paras["alpha"+str(ind)])
            out_beta_test = np.asarray(Dict_prob_paras["beta"+str(ind)])
            for ind,val in enumerate(xxSorted):
                index = np.where(val == xx)[0][0]
                alphaIntermediate = []
                piIntermediate = []
                betaIntermediate = []
                for i in range(out_beta_test.shape[1]):
                    alphaIntermediate.append(out_alpha_test[index,i])
                    betaIntermediate.append(out_beta_test[index,i])
                    piIntermediate.append(out_pi_test[index,i])
                alpha = np.sum(np.asarray(piIntermediate) * np.asarray(alphaIntermediate))
                betaP = np.sum(np.asarray(piIntermediate) * np.asarray(betaIntermediate))
                xxSorted.shape = (xxSize)
                Pr_xx[ind,:] = tf_beta(xxSorted, alpha, betaP)
                if  plotIndividualPDFs and ind%plotEvery == 0:
                    ax3 = fig3.add_subplot(nRows,nCols,plotCounter) 
                    plotCounter += 1
                    legendStr = []
                    for i in range(np.size(piIntermediate)):
                        if piIntermediate[i] >= np.max(piIntermediate)*1e-1:
                            yPDF = tf_beta(xxSorted, alphaIntermediate[i], betaIntermediate[i])
                            ax3.plot(xxSorted,yPDF)
                            legendStr.append("%.4f" % piIntermediate[i])
                    Pr_w = tf_beta(xxSorted, alpha,betaP)
                    #ax3.ticklabel_format(self, *, axis='both', style='sci')
                    ax3.plot(xxSorted, Pr_w,"--")
                    ax3.plot(val,max(Pr_w), "kx")
        if "Test" in title:
            self.Dict_Test["forFig_" + title + "_xxSorted"] = xxSorted
            self.Dict_Test["forFig_" + title + "_Pr_xx"] = Pr_xx
        
        for ind,val in enumerate(xxSorted):
            index = np.where(val == xx)[0][0]
            muBase = []
            sigmaBase = []
            for baseInd in range(numParameters):
                muBase.append(Dict_baseline["mu_var" + str(baseInd)][index])
                sigmaBase.append(Dict_baseline["variance_var" + str(baseInd)][index])
            muBase = dimensionalize(np.asarray(muBase),indPara)[0]
            sigmaBase = dimensionalize(np.asarray(sigmaBase),indPara,True)[0]
            #varMP = np.min(np.asarray(sigmaBase))
            #muMP = muBase[np.where(varMP == sigmaBase)[0][0]]
            prb = 0
            #weights = 1/len(sigmaBase)
            #for _i,_s in enumerate(sigmaBase):
                #prb += weights*norm.pdf(xxSorted, muBase[_i], _s)  
            prb = norm.pdf(xxSorted, muBase, sigmaBase)  
            prb.shape = (prb.shape[0],)
            Pr_baseline[ind,:] = prb 
            
        if "Test" in title:
            self.Dict_Test["forFig_" + title + "_averageSTD"] = np.mean(np.asarray(combined_mdn_std))
        
        print("Average standard deviation: " + str(np.mean(np.asarray(combined_mdn_std))))
        
        if "Ra" in title:
            titlep = "$\log(\eta_{ref})$ [Pa s]"
        elif "ERef" in title:
            titlep = r"$E$ [J mol$^{-1}$]"
        elif "VRef" in title:
            titlep = r"$V$ [m$^3$ mol$^{-1}$]"
        elif "Enrichment_cr" in title:
            titlep = "$\Lambda$"
        elif "iniTempTop" in title:
            titlep = "$T_{ini}$ [K]"
        
        x = np.zeros((np.size(xxSorted),np.size(xxSorted)))
        y = np.zeros((np.size(xxSorted),np.size(xxSorted)))
        for ind,val in enumerate(xxSorted):
            x[ind,:] = val
            y[:,ind] = xxSorted[ind]
        ax.contourf(x,y,Pr_xx)
        
        if "Test" in title:
            self.Dict_Test["forFig_" + title + "x"] = x
            self.Dict_Test["forFig_" + title + "y"] = y
        
        ax.set_xlabel("True")
        ax.set_ylabel("Predicted")
        ax.set_title(titlep + "; MDN") #, fontname="Times New Roman Bold") 
        #tex = r'$\bar{\sigma}$ = ' + str(np.mean(np.asarray(combined_mdn_std)))
        
        ax1.contourf(x,y,Pr_baseline)  
        ax1.set_xlabel("True")
        ax1.set_ylabel("Predicted")
        ax1.set_title(titlep + "; MP")#, fontname="Times New Roman Bold) 
        print()  
        if "Ra" in title:
            ax.set_xticks([19,20,21,22])
            ax.set_yticks([19,20,21,22])
            #ax.text(1, 1, tex, fontsize=20, va='bottom', color='white')
            ax1.set_xticks([19,20,21,22])
            ax1.set_yticks([19,20,21,22])
            
        if "ERef" in title:
            ax.set_xticks([1e+5,3e+5,5e+5])
            ax.set_yticks([1e+5,2e+5,3e+5,4e+5,5e+5])
            ax1.set_xticks([1e+5,3e+5,5e+5])
            ax1.set_yticks([1e+5,2e+5,3e+5,4e+5,5e+5])
            ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
            ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
            ax1.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
            ax1.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
        elif "VRef" in title:
            #ax.set_xscale('log')
            #ax.set_yscale('log')
            #ax1.set_xscale('log')
            #ax1.set_yscale('log')
            ax.set_xticks([4e-6, 7e-6, 10e-6])
            ax.set_yticks([4e-6, 5e-6, 6e-6, 7e-6, 8e-6, 9e-6, 10e-6])
            ax1.set_xticks([4e-6, 7e-6, 10e-6])
            ax1.set_yticks([4e-6, 5e-6, 6e-6, 7e-6, 8e-6, 9e-6, 10e-6])
            ax.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
            ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
            ax1.yaxis.set_major_formatter(plt.FuncFormatter(format_func))
            ax1.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
            
        elif "Enrichment_cr" in title:
            ax.set_xticks([1,10,20,30,40,50])
            ax.set_yticks([1,10,20,30,40,50])
            ax1.set_xticks([1,10,20,30,40,50])
            ax1.set_yticks([1,10,20,30,40,50])
        elif "iniTempTop" in title:
            ax.set_xticks([1600,1650,1700,1750,1800])
            ax.set_yticks([1600,1650,1700,1750,1800])
            ax1.set_xticks([1600,1650,1700,1750,1800])
            ax1.set_yticks([1600,1650,1700,1750,1800])

        for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
         ax.get_xticklabels() + ax.get_yticklabels()):
            item.set_fontsize(20)
            
        for item in ([ax1.title, ax1.xaxis.label, ax1.yaxis.label] +
         ax1.get_xticklabels() + ax1.get_yticklabels()):
            item.set_fontsize(20)

In [41]:
class MDN:
    def __init__(self, data, x_data,y_data, x_test,y_test, x_cv,y_cv, hSize, KMIX, NEPOCH, learnRate, \
                 paraVec, xparaVec, Dict_baseline_train, Dict_baseline_test, picID, repeats, id_string, \
                 kernel='gaussian',activation='tanh', multivar=False, multivarString="p_ra", trainORload='train'):
        print("Hidden Layers: " + str(hSize))
        print("Number of Mixtures: " + str(KMIX))
        print('Kernel is ' + str(kernel))
        print('Activation is ' + str(activation))
        
        print("X_train shape: " + str(x_data.shape))
        print("Y_train shape: " + str(y_data.shape))
        print("X_test shape: " + str(x_test.shape))
        print("Y_test shape: " + str(y_test.shape))
        print("X_cv shape: " + str(x_cv.shape))
        print("Y_cv shape: " + str(y_cv.shape))
        
        if multivar:
            print(multivarString)
        print()
        
        self.pMin = data.pMin
        self.pMax = data.pMax
        tf.reset_default_graph()
        yNumParameters = (y_data.shape)[1]
        numParameters = np.size(xparaVec)
        STDEV = 0.2
        MEAN = 0.
        xSize = (x_data.shape)[1] #size of each input
        
        # set up parameters
        W = []
        b = []
        layer = []

        x = tf.placeholder(dtype=tf.float64, shape=[None,xSize], name="x")
        y = tf.placeholder(dtype=tf.float64, shape=[None,yNumParameters], name="y")
        # first layer
        W.append(tf.Variable(tf.random_normal([xSize, hSize[0]], mean=MEAN, stddev=STDEV, dtype=tf.float64)))
        b.append(tf.Variable(tf.random_normal([1,hSize[0]], mean=MEAN, stddev=STDEV, dtype=tf.float64)))
        # add hidden layers (variable number)
        for i in range(1,len(hSize)):
            W.append(tf.Variable(tf.random_normal([hSize[i-1], hSize[i]], mean=MEAN, stddev=STDEV, dtype=tf.float64)))
            b.append(tf.Variable(tf.random_normal([1,hSize[i]], mean=MEAN, stddev=STDEV, dtype=tf.float64)))
            
        def pdf(x):
            return 1./np.sqrt(2.*np.pi) * tf.exp(-x**2/2.)
        #def cdf(x):
        #    return (1. + tf.erf(x/np.sqrt(2))) / 2.
        def skew(x,e,w,a):
            t = (x-e) / w
            return 2. / w * pdf(t) * cdf(a*t)
        def trapz(y, x):
            d = tf.subtract(x[1:],x[0:-1])
            return tf.reduce_sum((y[0:-1] + y[1:]) * d / 2.)
        def inner_function_gmc(out_piI):
            max_piI = tf.reduce_max(out_piI, 1, keepdims=True)
            out_piI = tf.subtract(out_piI, max_piI)
            out_piI = tf.exp(out_piI)
            normalize_piI = tf.reciprocal(tf.reduce_sum(out_piI, 1, keepdims=True))
            out_piI = tf.multiply(normalize_piI, out_piI)
            return out_piI
        def getOutput(W,b,layer,hSize):
            if activation=='relu':
                layer.append(tf.nn.relu(tf.matmul(x, W[0]) + b[0]))
                for i in range(1,len(hSize)):
                    layer.append(tf.nn.relu(tf.matmul(layer[i-1], W[i]) + b[i]))
            if activation=='leaky_relu':
                layer.append(tf.nn.leaky_relu(tf.matmul(x, W[0]) + b[0]))
                for i in range(1,len(hSize)):
                    layer.append(tf.nn.leaky_relu(tf.matmul(layer[i-1], W[i]) + b[i]))
            if activation=='elu':
                layer.append(tf.nn.elu(tf.matmul(x, W[0]) + b[0]))
                for i in range(1,len(hSize)):
                    layer.append(tf.nn.elu(tf.matmul(layer[i-1], W[i]) + b[i]))
            if activation=='tanh':
                layer.append(tf.nn.tanh(tf.matmul(x, W[0]) + b[0]))
                for i in range(1,len(hSize)):
                    layer.append(tf.nn.tanh(tf.matmul(layer[i-1], W[i]) + b[i]))
            return tf.matmul(layer[-1],W[-1]) + b[-1]  
        def get_mixture_coef3(output):
            out_pi = tf.placeholder(dtype=tf.float64, shape=[None,KMIX* yNumParameters], name="mixparam")
            out_sigma = tf.placeholder(dtype=tf.float64, shape=[None,KMIX* yNumParameters], name="mixparam")
            out_mu = tf.placeholder(dtype=tf.float64, shape=[None,KMIX* yNumParameters], name="mixparam")
            out_pi, out_sigma, out_mu = tf.split(output,3,1)
            out_piI = inner_function_gmc((tf.split(out_pi, yNumParameters, 1))[0])
            for ind in range(1,yNumParameters):
                out_piI = tf.concat([out_piI, inner_function_gmc((tf.split(out_pi, yNumParameters, 1))[ind])], 1)
            out_pi = out_piI
            return out_pi, out_sigma, out_mu
        
        def get_mixture_coef3B(output):
            out_pi = tf.placeholder(dtype=tf.float64, shape=[None,KMIX* yNumParameters], name="mixparam")
            out_alpha = tf.placeholder(dtype=tf.float64, shape=[None,KMIX* yNumParameters], name="mixparam")
            out_beta = tf.placeholder(dtype=tf.float64, shape=[None,KMIX* yNumParameters], name="mixparam")
            out_pi, out_alpha, out_beta = tf.split(output,3,1)
            out_piI = inner_function_gmc((tf.split(out_pi, yNumParameters, 1))[0])
            for ind in range(1,yNumParameters):
                out_piI = tf.concat([out_piI, inner_function_gmc((tf.split(out_pi, yNumParameters, 1))[ind])], 1)
            out_pi = out_piI
            return out_pi, out_alpha, out_beta
        
        if kernel == 'skewed_gaussian':
            ySize = KMIX * 4 # pi, mu, stdev, skew
            # add final layer
            W.append(tf.Variable(tf.random_normal([hSize[-1], ySize * yNumParameters], mean=MEAN, stddev=STDEV, dtype=tf.float64)))
            b.append(tf.Variable(tf.random_normal([1,ySize * yNumParameters], mean=MEAN, stddev=STDEV, dtype=tf.float64)))
            # define model
            output = getOutput(W,b,layer,hSize)

            def get_mixture_coef(output):
                out_pi = tf.placeholder(dtype=tf.float64, shape=[None,KMIX* yNumParameters], name="mixparam")
                out_sigma = tf.placeholder(dtype=tf.float64, shape=[None,KMIX* yNumParameters], name="mixparam")
                out_mu = tf.placeholder(dtype=tf.float64, shape=[None,KMIX* yNumParameters], name="mixparam")
                out_skew = tf.placeholder(dtype=tf.float64, shape=[None,KMIX* yNumParameters], name="mixparam")
                out_pi, out_sigma, out_mu, out_skew = tf.split(output,4,1)
                out_piI = inner_function_gmc((tf.split(out_pi, yNumParameters, 1))[0])
                for ind in range(1,yNumParameters):
                    out_piI = tf.concat([out_piI, inner_function_gmc((tf.split(out_pi, yNumParameters, 1))[ind])], 1)
                out_pi = out_piI
                out_sigma = tf.exp(out_sigma)
                out_mu = out_mu
                return out_pi, out_sigma, out_mu, out_skew

            out_pi, out_sigma, out_mu, out_skew = get_mixture_coef(output)
            oneDivSqrtTwoPI = 1. / np.sqrt(2*np.pi) # normalisation factor for gaussian
   
            def tf_skew(y, mu, sigma, a):
                result = skew(y,mu,sigma,a)
                return result 
            def get_lossfunc(out_pi, out_sigma, out_mu, out_skew, y):
                yI = (tf.split(y,yNumParameters,1))[0]
                out_muI = (tf.split(out_mu,yNumParameters,1))[0]
                out_sigmaI = (tf.split(out_sigma,yNumParameters,1))[0]
                out_skewI = (tf.split(out_skew,yNumParameters,1))[0]
                resultI = tf_skew(yI, out_muI, out_sigmaI, out_skewI)
                for ind in range(1,yNumParameters):
                    yI = (tf.split(y,yNumParameters,1))[ind]
                    out_muI = (tf.split(out_mu,yNumParameters,1))[ind]
                    out_sigmaI = (tf.split(out_sigma,yNumParameters,1))[ind]
                    out_skewI = (tf.split(out_skew,yNumParameters,1))[ind]
                    resultII = tf_skew(yI, out_muI, out_sigmaI, out_skewI)
                    resultI = tf.concat([resultI, resultII], 1)
                result = resultI
                result = tf.multiply(result, out_pi)
                result = tf.reduce_sum(result, 1, keepdims=True)
                result = -tf.log(result)
                return tf.reduce_mean(result)

            lossfunc = get_lossfunc(out_pi, out_sigma, out_mu, out_skew, y)
            train_op = tf.train.AdamOptimizer(learning_rate=learnRate).minimize(lossfunc)
        
        elif kernel=='gaussian':
            ySize = KMIX * 3 # pi, mu, stdev
            # add final layer
            W.append(tf.Variable(tf.random_normal([hSize[-1], ySize * yNumParameters], stddev=STDEV, dtype=tf.float64)))
            b.append(tf.Variable(tf.random_normal([1,ySize * yNumParameters], stddev=STDEV, dtype=tf.float64)))
            # define model
            output = getOutput(W,b,layer,hSize)   
           
            out_pi, out_sigma, out_mu = get_mixture_coef3(output)
            out_sigma = tf.exp(out_sigma)
            oneDivSqrtTwoPI = 1. / np.sqrt(2*np.pi) # normalisation factor for gaussian

            def tf_normal(y, mu, sigma):
                result = tf.subtract(y, mu)
                result = tf.multiply(result,tf.reciprocal(sigma))
                result = -tf.square(result)/2.
                result = tf.multiply(tf.multiply(tf.exp(result),tf.reciprocal(sigma)),oneDivSqrtTwoPI)
                return result 
            def get_lossfunc(out_pi, out_sigma, out_mu, y):
                yI = (tf.split(y,yNumParameters,1))[0]
                out_muI = (tf.split(out_mu,yNumParameters,1))[0]
                out_sigmaI = (tf.split(out_sigma,yNumParameters,1))[0]
                resultI = tf_normal(yI, out_muI, out_sigmaI)
                for ind in range(1,yNumParameters):
                    yI = (tf.split(y,yNumParameters,1))[ind]
                    out_muI = (tf.split(out_mu,yNumParameters,1))[ind]
                    out_sigmaI = (tf.split(out_sigma,yNumParameters,1))[ind]
                    resultII = tf_normal(yI, out_muI, out_sigmaI)
                    resultI = tf.concat([resultI, resultII], 1)
                result = resultI
                result = tf.multiply(result, out_pi)
                result = tf.reduce_sum(result, 1, keepdims=True)
                result = -tf.log(result)
                return tf.reduce_mean(result)
            lossfunc = get_lossfunc(out_pi, out_sigma, out_mu, y) + 0.01*tf.add_n([tf.nn.l2_loss(w) for w in W])
            train_op = tf.train.AdamOptimizer(learning_rate=learnRate).minimize(lossfunc)
            
        elif kernel=='beta':
            ySize = KMIX * 3 # pi, alpha, beta
            # add final layer
            W.append(tf.Variable(tf.random_normal([hSize[-1], ySize * yNumParameters], mean=MEAN, \
                                                  stddev=STDEV, dtype=tf.float64)))
            b.append(tf.Variable(tf.random_normal([1,ySize * yNumParameters], mean=MEAN, \
                                                  stddev=STDEV, dtype=tf.float64)))
            # define model
            output = getOutput(W,b,layer,hSize)   
            out_pi, out_alpha, out_beta = get_mixture_coef3B(output)
            def getGamma(z):
                return tf.exp(tf.lgamma(tf.subtract(z,1)))
            def tf_beta(y, alpha, beta):
                sumIs = tf.add(alpha,beta)
                Z = tf.divide(tf.multiply(getGamma(alpha),getGamma(beta)), getGamma(sumIs))
                result = tf.divide(tf.multiply(tf.pow(y,(alpha - 1.)), tf.pow((1. - y),(beta - 1.))),Z)
                return  result
            def get_lossfunc(out_pi, out_alpha, out_beta, y):
                yI = (tf.split(y,yNumParameters,1))[0]
                out_alphaI = (tf.split(out_alpha,yNumParameters,1))[0]
                out_betaI = (tf.split(out_beta,yNumParameters,1))[0]
                resultI = tf_beta(yI, out_alphaI, out_betaI)
                for ind in range(1,yNumParameters):
                    yI = (tf.split(y,yNumParameters,1))[ind]
                    out_alphaI = (tf.split(out_alpha,yNumParameters,1))[ind]
                    out_betaI = (tf.split(out_beta,yNumParameters,1))[ind]
                    resultII = tf_beta(yI, out_alphaI, out_betaI)
                    resultI = tf.concat([resultI, resultII], 1)
                result = resultI
                result = tf.multiply(result, out_pi)
                result = tf.abs(tf.reduce_sum(result, 1, keepdims=True))
                result = -tf.log(result)
                result = tf.reduce_mean(result)
                return result 
            lossfunc = get_lossfunc(out_pi, out_alpha, out_beta, y)
            train_op = tf.train.AdamOptimizer(learning_rate=learnRate).minimize(lossfunc)
            
        lossCheck = np.nan
        trials = 0        
        lossList = []
        loss_cvList = []
        trackLoss = True
        
        sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
        saver = tf.train.Saver()
        if trainORload=='train':
            while (np.isnan(lossCheck) and trials < 20) or trials < repeats:
                #sess = tf.InteractiveSession()
                sess.run(tf.global_variables_initializer())
                #print(sess.run(output, feed_dict={x: x_data, y: y_data}))

                loss = [] #np.zeros(NEPOCH) # store the training progress here.
                loss_cv = []
                loss_testList = []
                sess.run(train_op,feed_dict={x: x_data, y: y_data})
                loss.append(sess.run(lossfunc, feed_dict={x: x_data, y: y_data}))
                loss_cv.append(sess.run(lossfunc, feed_dict={x: x_cv, y: y_cv}))

                sess.run(train_op,feed_dict={x: x_data, y: y_data})
                loss.append(sess.run(lossfunc, feed_dict={x: x_data, y: y_data}))
                loss_cv.append(sess.run(lossfunc, feed_dict={x: x_cv, y: y_cv}))
                i = 1  
                while loss_cv[i] <= loss_cv[i-1]:
                    sess.run(train_op,feed_dict={x: x_data, y: y_data})
                    loss.append(sess.run(lossfunc, feed_dict={x: x_data, y: y_data}))
                    loss_cv.append(sess.run(lossfunc, feed_dict={x: x_cv, y: y_cv}))
                    i += 1
                    if trackLoss and i%10000==0:
                        print(i,loss[-1],loss_cv[-1])

                while loss_cv[i] <= loss_cv[i-int(np.floor(NEPOCH*i/100))]:
                    sess.run(train_op,feed_dict={x: x_data, y: y_data})
                    loss.append(sess.run(lossfunc, feed_dict={x: x_data, y: y_data}))
                    loss_cv.append(sess.run(lossfunc, feed_dict={x: x_cv, y: y_cv}))
                    i += 1
                    if trackLoss and i%10000==0:
                        print(i,loss[-1],loss_cv[-1])

                lossList.append(loss[-1])
                loss_cvList.append(loss_cv[-1])
                lossCheck = loss[-1]
                trials += 1

                loss_testList.append(sess.run(lossfunc, feed_dict={x: x_test, y: y_test}))

                print("Training loss dropped to " + str(lossCheck) + "in trial " + str(trials))
                print("CV loss dropped to " + str(loss_cv[-1]) + "in trial " + str(trials))
            
                #saver.save(sess, pathSave + "TrainedNetworks/" + id_string + "/MDN_trial_" + str(trials))
            print('--------------------------------------------------------------------------')
            lossTrainStore_mean = np.mean(np.asarray(lossList))
            lossTrainStore_std = np.std(np.asarray(lossList))
            lossTestStore_mean = np.mean(np.asarray(loss_testList))
            lossTestStore_std = np.std(np.asarray(loss_testList))
            print("Train loss is " + str(lossTrainStore_mean) + ' +/- ' + str(lossTrainStore_std))
            print("Test loss is " + str(lossTestStore_mean) + ' +/- ' + str(lossTestStore_std))
            print('--------------------------------------------------------------------------')
        else:
            saver.restore(sess, pathSave + "TrainedNetworks/" + id_string + "/MDN_trial_0")
                
        Dict_Train = {}
        Dict_Test = {}
        Dict_CV = {}

        def generateDict(x_I, yNumParameters):
            Dict_I = {}
            if kernel=='skewed_gaussian':
                out_pi_test, out_sigma_test, out_mu_test, out_skew_test = sess.run(get_mixture_coef(output), feed_dict={x: x_I})   
                for i in range(yNumParameters):
                    Dict_I["sigma" + str(i)] = (np.split(out_sigma_test, yNumParameters, axis=1))[i]
                    Dict_I["mu" + str(i)] = (np.split(out_mu_test, yNumParameters, axis=1))[i]
                    Dict_I["pi" + str(i)] = (np.split(out_pi_test, yNumParameters, axis=1))[i]
                    Dict_I["skew" + str(i)] = (np.split(out_skew_test, yNumParameters, axis=1))[i]
            elif kernel=='gaussian':
                out_pi_test, out_sigma_test, out_mu_test = sess.run(get_mixture_coef3(output), feed_dict={x: x_I}) 
                out_sigma_test = np.exp(out_sigma_test)
                for i in range(yNumParameters):
                    Dict_I["sigma" + str(i)] = (np.split(out_sigma_test, yNumParameters, axis=1))[i]
                    Dict_I["mu" + str(i)] = (np.split(out_mu_test, yNumParameters, axis=1))[i]
                    Dict_I["pi" + str(i)] = (np.split(out_pi_test, yNumParameters, axis=1))[i]   
            elif kernel=='beta':
                out_pi_test, out_alpha_test, out_beta_test = sess.run(get_mixture_coef3(output), feed_dict={x: x_I})   
                for i in range(yNumParameters):
                    Dict_I["alpha" + str(i)] = (np.split(out_alpha_test, yNumParameters, axis=1))[i]
                    Dict_I["beta" + str(i)] = (np.split(out_beta_test, yNumParameters, axis=1))[i]
                    Dict_I["pi" + str(i)] = (np.split(out_pi_test, yNumParameters, axis=1))[i]   
            return Dict_I

        alphaVal = 0.5
        Dict_Train = generateDict(x_data, yNumParameters)
        Dict_Test = generateDict(x_test, yNumParameters)

        for ind in range(yNumParameters):
            fig = plt.figure(figsize=(14,9), dpi=200)
            plt.subplots_adjust(top=0.90, bottom=0.08, left=0.10, right=0.95, hspace=0.3,wspace=0.4)
            nCols = 3 # 2
            nRows = 2 #yNumParameters//nCols + 2
            plotCounter = 1        
            Dict_Test["forFig_" + paraVec[ind] + "_loss"] = loss
            Dict_Test["forFig_" + paraVec[ind] + "_loss_cv"] = loss_cv
            ax = fig.add_subplot(nRows,nCols,plotCounter) 
            ax.plot(loss, 'r-', label='Training')
            ax.plot(loss_cv,"b--", label='Validation')
            ax.set_xlabel('Epochs')
            ax.set_ylabel('Negative log-likelihood')
            #plt.title('Loss')
            ax.legend()
            #ax.set_xticks([0, 50000, 100000])
            ax.locator_params(axis='x', nbins=3)
            plotCounter += 1
            for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
            ax.get_xticklabels() + ax.get_yticklabels()):
                item.set_fontsize(20)
            ax = fig.add_subplot(nRows,nCols,plotCounter)
            plotCounter += 1
            ax1 = fig.add_subplot(nRows,nCols,plotCounter)
            plotCounter += 2

            pR = plotResults(data,ax,ax1, alphaVal,(paraVec[ind] + ", Train"), y_data[:,ind], Dict_Train, \
                             ind, Dict_baseline_train, numParameters, kernel, False)
            Dict_Train["loss_train_mean"] = lossTrainStore_mean
            Dict_Train["loss_train_std"] = lossTrainStore_std
            ax = fig.add_subplot(nRows,nCols,plotCounter)
            plotCounter += 1
            ax1 = fig.add_subplot(nRows,nCols,plotCounter)
            plotCounter += 1
            pRT = plotResults(data,ax,ax1, alphaVal,(paraVec[ind] + ", Test"), y_test[:,ind], Dict_Test, \
                        ind, Dict_baseline_test, numParameters, kernel, True)
            Dict_Test = pRT.Dict_Test
            Dict_Test["loss_test_mean"] = lossTestStore_mean
            Dict_Test["loss_test_std"] = lossTestStore_std
            #fig.savefig(pathSave + "inverseProblemPics/" + "Output_fig" + str(picID) + ".pdf", bbox_inches='tight')

            fig.tight_layout()
        plt.show()
        print("-------------------------------------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------------------------------------")
        print()
        sess.close()

        self.Dict_Train = Dict_Train
        self.Dict_Test = Dict_Test

In [42]:
class meanPredictor:
    def __init__(self, x_data,y_data,xparaVec, plotMP=False):
        Dict_Mean_Predictor = {}
        if 'Tprof_4p5Gyr' in xparaVec and np.size(xparaVec)==1:
            tProfOnly = True
        else:
            tProfOnly = False
        for i in range(np.size(xparaVec)):
            if tProfOnly or (xparaVec[i] == 'Tprof_4p5Gyr'):
                observable = np.mean(x_data,axis=1)
            else:
                observable = x_data[:,i] 
            para = y_data[:,0]
            steps = 8
            
            bin_means, bin_edges, binnumber = binned_statistic(observable, para, statistic='mean',bins=steps)
            bin_variance = [np.std(para[np.where(binnumber == bIndex)[0]]) for bIndex in range(1,steps+1)]
            removeVec = []
            notRemoveVec = []
            for i3 in range(np.size(bin_means)):
                if np.isnan(bin_means[i3]) or bin_variance[i3] == 0.0:
                    removeVec.append(i3)
                else:
                    notRemoveVec.append(i3)

            for r in removeVec:
                bin_variance[r] = np.average([bin_variance[notInd] for notInd in notRemoveVec])
                bin_means[r] = np.average([bin_means[notInd] for notInd in notRemoveVec])

            digitizer = np.digitize(observable,bin_edges) - 1
            mu = np.zeros(np.size(digitizer))
            variance = np.zeros(np.size(digitizer))
            for ind in range(np.size(digitizer)):
                mu[ind] = bin_means[digitizer[ind]-1]
                variance[ind] = bin_variance[digitizer[ind]-1]
            Dict_Mean_Predictor["mu_var" + str(i)] = mu
            Dict_Mean_Predictor["variance_var" + str(i)] = variance
            if plotMP:
                plt.figure(figsize=(4,6))
                plt.plot(observable,para, "ro", alpha = 0.4)
                plt.plot((bin_edges[1:]+bin_edges[0:-1])/2.,bin_means, "k-")
                plt.plot((bin_edges[1:]+bin_edges[0:-1])/2.,bin_means+bin_variance,"b--")
                plt.plot((bin_edges[1:]+bin_edges[0:-1])/2.,bin_means-bin_variance,"b--")
                plt.legend([_, "$\mu$", "$\mu + \sigma$", "$\mu - \sigma$"], loc="lower_right")
        self.Dict_Mean_Predictor = Dict_Mean_Predictor

In [45]:
def bigLoop(_zipped, repeats=5, multivar=False):
    Dict_kls = {}
    hSize, KMIX, x_o, x_b, trainPercent, yIndex, kernel, activation, noiseLevel = _zipped
    fontSize = 14
    picID = 0 #yIndices  [yIndices[i5]]
    data = getData([yIndex], x_o, x_b, trainPercent, pathSave, noiseLevel)
    
    id_string = str(_zipped)
    with open(pathSave + "/Data_Files/processedData" + id_string + ".txt", "rb") as fkl:
        dataDict = load(fkl)
    
    data.x_data = dataDict['x_data']
    data.x_test = dataDict['x_test']
    data.x_cv = dataDict['x_cv']     
    data.y_data = dataDict['y_data']
    data.y_test = dataDict['y_test']
    data.y_cv = dataDict['y_cv'] 
    data.paraVec = dataDict['paraVec']
    data.xparaVec = dataDict['xparaVec']
    data.rProf = dataDict['rProf']
    data.startPoint = dataDict['startPoint']
    data.endPoint = dataDict['endPoint']    
    data.indexer = dataDict['indexer']    
    data.pMax = dataDict['pMax']  
    data.pMin = dataDict['pMin']  
    data.oMax = dataDict['oMax']
    data.oMin = dataDict['oMin']
    
    trainORload = 'train'
    
    Dict_MP_Test = meanPredictor(data.x_test, data.y_test, data.xparaVec, False)
    Dict_MP_Train = meanPredictor(data.x_data, data.y_data, data.xparaVec, False)

    #data.visualizeData(data, picID, fontSize, Dict_MP_Train.Dict_Mean_Predictor)

    learn_rate = 0.1
    Dict_MDN = MDN(data,data.x_data,data.y_data, data.x_test,data.y_test, data.x_cv, \
                       data.y_cv, hSize, KMIX, 1, learn_rate, \
                       data.paraVec, data.xparaVec, Dict_MP_Train.Dict_Mean_Predictor, \
                       Dict_MP_Test.Dict_Mean_Predictor, picID, repeats, id_string, kernel, activation, False, 
                       "p_ra", trainORload)

    Dict_kls[id_string + "Loss_train_mean"] = Dict_MDN.Dict_Train["loss_train_mean"]
    Dict_kls[id_string + "Loss_train_std"] = Dict_MDN.Dict_Train["loss_train_std"]
    Dict_kls[id_string + "Loss_test_mean"] = Dict_MDN.Dict_Test["loss_test_mean"]
    Dict_kls[id_string + "Loss_test_std"] = Dict_MDN.Dict_Test["loss_test_std"]

    picID += 1

In [ ]:
#Parameter indices: 0-Rayleigh number, 4-activation energy, 5-activation volume, 6-enrichment factor, 7-initial temperature
#Obsbervables indices: 0-Q_c, 1-Q_s, 2-R_th, 3-D_e, 4-D_melt, 5-t_volc, 6-T_profs

# hidden layers, mixtures, observables, bounds for temperature profile, size of training dataset, parameter, kernel, activation, Noise level
zipped_new = [[[12, 6], 3, [0, 1, 2, 3, 4, 5, 6], [0, 100], 0.8, 0, 'gaussian', 'tanh', 0.0],
              [[12, 6], 3, [0, 1, 2, 3, 4, 5, 6], [0, 100], 0.8, 4, 'gaussian', 'tanh', 0.0],
              [[12, 6], 3, [0, 1, 2, 3, 4, 5, 6], [0, 100], 0.8, 5, 'gaussian', 'tanh', 0.0],
              [[12, 6], 3, [0, 1, 2, 3, 4, 5, 6], [0, 100], 0.8, 6, 'gaussian', 'tanh', 0.0],
              [[12, 6], 3, [0, 1, 2, 3, 4, 5, 6], [0, 100], 0.8, 7, 'gaussian', 'tanh', 0.0]
             ]
runParallel = False
if runParallel:                   
    Parallel(n_jobs=1, verbose=10, backend='loky', prefer='processes')(delayed(bigLoop)(_z, 1, True) for _z in zipped_new)
    
else:
    [bigLoop(_z) for _z in zipped_new] 